In [1]:
import os, re, json, sys, time, requests, argparse
import cmipld
from cmipld.utils.git import tree

In [16]:
tree.get_repo_tree, cmipld.mapping

(<function cmipld.utils.git.tree.get_repo_tree(owner, repo, path='src-data', branch='main', prefix=None)>,
 {'wcrp-universe': 'https://wcrp-cmip.github.io/WCRP-UNIVERSE/',
  'mip-variables': 'https://wcrp-cmip.github.io/MIP-variables/',
  'cmip6plus': 'https://wcrp-cmip.github.io/CMIP6Plus_CVs/',
  'cmip7': 'https://wcrp-cmip.github.io/CMIP7_CVs/',
  'cf': 'https://wcrp-cmip.github.io/CF/',
  'obs4mips': 'https://wolfiex.github.io/obs4MIPs-cmor-tables-ld//'})

In [17]:
root = {
    'name': 'cmipld',
    "html_url":f"https://wcrp-cmip.org",
    "children": []
}

import time
for k,v in cmipld.mapping.items():
    try:
        
        owner,repo = v.replace('https://','').split('/')[:2]
        owner = owner.split('.')[0]
        
        
        print(owner,repo)
        
        entry = tree.get_repo_tree(owner, repo, path="src-data", branch = "production", prefix = k)

        
        root['children'].append(entry)
        time.sleep(1)
        
        
    except Exception as e:
        print(k,v)
        print(e)
        print('---')
        
        
# json.dump(root, open('universe_hierarchy.json','w'), indent=2)

wcrp-cmip WCRP-UNIVERSE
https://api.github.com/repos/wcrp-cmip/WCRP-UNIVERSE/contents/src-data?ref=production
wcrp-cmip MIP-variables
https://api.github.com/repos/wcrp-cmip/MIP-variables/contents/src-data?ref=production
wcrp-cmip CMIP6Plus_CVs
https://api.github.com/repos/wcrp-cmip/CMIP6Plus_CVs/contents/src-data?ref=production
wcrp-cmip CMIP7_CVs
https://api.github.com/repos/wcrp-cmip/CMIP7_CVs/contents/src-data?ref=production
wcrp-cmip CF
https://api.github.com/repos/wcrp-cmip/CF/contents/src-data?ref=production
wolfiex obs4MIPs-cmor-tables-ld
https://api.github.com/repos/wolfiex/obs4MIPs-cmor-tables-ld/contents/src-data?ref=production


In [4]:
# entry = tree.get_repo_tree(owner, repo, path="src-data", branch = "production")

In [5]:
# repo = 'CMIP6Plus_CVs'
# owner = 'wcrp-cmip'
# url = f"https://api.github.com/repos/{owner}/{repo}/contents/"

        
# owner,repo = url.replace('https://api.github.com/repos/','').split('/',2)[:2]
# preurl = f"https://{owner}.github.io/{repo}/"


# owner, repo, preurl


In [19]:
root2 = tree.filter_nested_dict({**root},valid_extensions=[".json"],valid_names=[])
for e in root2['children']:
    e['name'] = e['prefix']

json.dump(root2, open('universe_hierarchy.json','w'), indent=2)

In [7]:
# root2

In [ ]:
g = []
from pyld import jsonld
for k,v in cmipld.mapping.items():
    print(k)
    g.extend(jsonld.expand(v+'graph.jsonld'))
    
json.dump(g, open('universe_graph.json','w'), indent=2)


wcrp-universe
mip-variables
cmip6plus
cmip7
cf
obs4mips


In [9]:
# links = [i for i in  if i['subject']['type']== 'IRI' and i['object']['type']== 'IRI']

In [ ]:
links = []
rdf = jsonld.to_rdf(g)

for i in rdf:
    i = rdf[i]
    if not i:
        continue
    for k in i:
        if k['subject']['type']== 'IRI' and k['object']['type']== 'IRI':
            # links.append(f"{k['subject']['value']}>{k['object']['value']}")
            links.append([k['subject']['value'],k['object']['value']])
            
with open ('universe_links.json','w') as f:
    f.write(f'const links = {json.dumps(links)};')

In [ ]:
# # from collections import Counter
# # Counter(links).most_common(10)

# links

In [23]:
structure = {}
for repo in root2['children']:
    structure[repo['prefix']] = {}
    for file in repo['children']:
        structure[repo['prefix']][file['name']] = file['jsonld']